<h1>Modelos</h1>

En esta versión del script, solo corremos los modelos considerando colegios urbanos y excluyendo NSE_puntaje, excepto que se aclare lo contrario

Finalmente, se calculan los valores shap

# 1. Librerías

In [1]:
#Generales
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import warnings
from collections import defaultdict
from datetime import date, datetime
from functools import reduce
import joblib 
import re
from tqdm.auto import tqdm
from contextlib import contextmanager
# Encoders
from sklearn.preprocessing import OneHotEncoder

# Métricas de performance de modelos
from sklearn.metrics import (mean_absolute_error,mean_squared_error, precision_score, r2_score)

# Modelos y entrenamiento
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import  RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import (
    GridSearchCV, ParameterGrid, RepeatedKFold, RandomizedSearchCV,
    cross_val_score, train_test_split, KFold)

# 2. Datos

Importamos la base de colegios que considerando resultados tanto en lengua como en matemática, eliminando las columnas que no son útiles para el modelado. En total, son 11.357 colegios

In [2]:
df = pd.read_excel('../outputs/dataset_final_colegios_y_variables.xlsx', sheet_name='data')
df = df.set_index('ID1', drop=True)  # Mantenemos ID1 como índice

info_extra = df[['jurisdiccion', 'numero_alumnos', 'NSE_puntaje', 'not_missing_matematica_ni_lengua']].drop_duplicates()

In [3]:
df.columns.to_list()

['porcentaje_satisfactorio',
 'numero_alumnos',
 'jurisdiccion',
 'sector',
 'ambito',
 'secciones_por_colegio',
 'region',
 'NSE_puntaje',
 'personas_por_habitacion',
 'clima_escolar',
 'ap27d_1_pct',
 'ap27d_2_pct',
 'ap27d_3_pct',
 'ap27d_4_pct',
 'ap16c_1_pct',
 'ap05a_1_pct',
 'ap05e_1_pct',
 'ap09d_1_pct',
 'ap17b_1_pct',
 'ap27a_1_pct',
 'ap27a_2_pct',
 'ap27a_3_pct',
 'ap27a_4_pct',
 'Nivel_Ed_Madre_1.0_pct',
 'Nivel_Ed_Madre_2.0_pct',
 'Nivel_Ed_Madre_3.0_pct',
 'Nivel_Ed_Madre_4.0_pct',
 'Nivel_Ed_Madre_5.0_pct',
 'Nivel_Ed_Madre_6.0_pct',
 'Nivel_Ed_Madre_7.0_pct',
 'ap03_1_pct',
 'ap03_2_pct',
 'ap03_3_pct',
 'migracion_1.0_pct',
 'ap09e_1_pct',
 'ap06c_1_pct',
 'ap06c_2_pct',
 'ap06c_3_pct',
 'ap06c_4_pct',
 'ap12_1_pct',
 'ap05c_1_pct',
 'ap27b_1_pct',
 'ap27b_2_pct',
 'ap27b_3_pct',
 'ap27b_4_pct',
 'ap21_1_pct',
 'ap21_2_pct',
 'ap21_3_pct',
 'ap21_4_pct',
 'sobreedad_0.0_pct',
 'sobreedad_1.0_pct',
 'ap27c_1_pct',
 'ap27c_2_pct',
 'ap27c_3_pct',
 'ap27c_4_pct',
 'ap22_

In [5]:
print(df.shape)

(11357, 308)


Importamos archivos con nombres y descripción de variables y respuestas

In [6]:
df_variables = pd.read_excel('../outputs/dataset_final_colegios_y_variables.xlsx',
    sheet_name='variables'
) 

Esta son las variables computadas como NSE. Distinguirlas resulta útil para luego hacer el análisis descriptivo de los resultados

In [7]:
NSE_base = list(set([
    "NSE_puntaje",
    "ap09a",
    "ap09d",
    "ap09k",
    "ap09g",
    "ap20b1",
    "ap20a2",
    "ap09j",
    "Nivel_Ed_Padre",
    "ap09c",
    "ap09i",
    "ap09b",
    "ap09h",
    "Nivel_Ed_Madre",
    "ap09e",
    "ap20b2",
    "ap09f",
    "ap20a1"
]))

NSE_todas_las_variables = [
    "NSE_puntaje",
    "ap09a_1_pct", "ap09a_2_pct",
    "ap09d_1_pct", "ap09d_2_pct",
    "ap09k_1_pct", "ap09k_2_pct",
    "ap09g_1_pct", "ap09g_2_pct",
    "ap20b1_1.0_pct", "ap20b1_2.0_pct",
    "ap20a2_1.0_pct", "ap20a2_2.0_pct", "ap20a2_3.0_pct", "ap20a2_4.0_pct",
    "ap09j_1_pct", "ap09j_2_pct",
    "Nivel_Ed_Padre_1.0_pct", "Nivel_Ed_Padre_2.0_pct", "Nivel_Ed_Padre_3.0_pct",
    "Nivel_Ed_Padre_4.0_pct", "Nivel_Ed_Padre_5.0_pct", "Nivel_Ed_Padre_6.0_pct", "Nivel_Ed_Padre_7.0_pct",
    "ap09c_1_pct", "ap09c_2_pct",
    "ap09i_1_pct", "ap09i_2_pct",
    "ap09b_1_pct", "ap09b_2_pct",
    "ap09h_1_pct", "ap09h_2_pct",
    "Nivel_Ed_Madre_1.0_pct", "Nivel_Ed_Madre_2.0_pct", "Nivel_Ed_Madre_3.0_pct",
    "Nivel_Ed_Madre_4.0_pct", "Nivel_Ed_Madre_5.0_pct", "Nivel_Ed_Madre_6.0_pct", "Nivel_Ed_Madre_7.0_pct",
    "ap09e_1_pct", "ap09e_2_pct",
    "ap20b2_1.0_pct", "ap20b2_2.0_pct",
    "ap09f_1_pct", "ap09f_2_pct",
    "ap20a1_1.0_pct", "ap20a1_2.0_pct", "ap20a1_3.0_pct", "ap20a1_4.0_pct",
    "personas_por_habitacion"
]

df_variables["NSE"] = np.where(df_variables["variable"].isin(NSE_base), 1, 0)

### 2.1 Preparación de datos para modelos

In [8]:
# ========= Definición de la x y la y  ========= #

columnas_a_droppear = ['porcentaje_satisfactorio', 'ambito', 'numero_alumnos', 'not_missing_matematica_ni_lengua', 'NSE_puntaje']
X = df.drop(columns=columnas_a_droppear)
y = df[['porcentaje_satisfactorio']]
sample_weight = df["not_missing_matematica_ni_lengua"]

# ========= Dividimos la muestra en train y test  ========= #
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
    X, y, sample_weight, test_size=0.2, random_state=1
)

# ========= Corremos el encoding para las categoricas  ========= #
x_cat = X_train.select_dtypes(include=['object']).columns.to_list()
x_cat.append("clima_escolar")

preprocessor = ColumnTransformer(
    [('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False , drop=None), x_cat)],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform="pandas")


# ========= Transformación  ========= #
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep = preprocessor.transform(X_test)

## La y queda tal cual, es solo para homogeneizar nombres. El squeeze es  para que sea serie (no df) pero sin perder el ID
y_train_prep=y_train.copy().squeeze()   
y_test_prep=y_test.copy().squeeze()

# 3. Modelos

## 3.1 Random Forest con pesos (w=número alumnos que responideron el examen)

Tenemos cuatro variables categoricas: jurisdiccion, región, sector (público/privado) y clima escolar. Usamos One hot encoding para generar variables dummies.

Probamos modelos con pesos, donde el ponderador es la cantidad de alumnos que respondieron el examen.

Probamos un random forest con hiperparámetros por default (los guardamos) y con hiperparámetros tuneados

Hacemos el feature importance en función de la reducción de la función de pérdida y luego haciendo shap values


### 3.1.1 Random Forest hiperparámetros por default

In [9]:
# ========= Iniciamos el modelo y entrenamos  ========= #
model= RandomForestRegressor(random_state=1, n_jobs=-1)
model.fit(X_train_prep, y_train_prep,  sample_weight=w_train)

RandomForestRegressor(n_jobs=-1, random_state=1)

In [10]:
# ========= Predecimos en el conjunto de train y de test  ========= #
y_train_pred = model.predict(X_train_prep)
y_test_pred = model.predict(X_test_prep)

In [11]:
# ========= Evaluamos la predicción  ========= #
print("Evaluación en train set:\n")
mae = mean_absolute_error(y_train_prep, y_train_pred, sample_weight=w_train)
rmse = np.sqrt(mean_squared_error(y_train_prep, y_train_pred, sample_weight=w_train))
r2 = r2_score(y_train_prep, y_train_pred, sample_weight=w_train)

print(f"  MAE : {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²  : {r2:.4f}\n")

print("")

print("Evaluación en test set:\n")

mae = mean_absolute_error(y_test_prep, y_test_pred, sample_weight=w_test)
rmse = np.sqrt(mean_squared_error(y_test_prep, y_test_pred, sample_weight=w_test))
r2 = r2_score(y_test_prep, y_test_pred, sample_weight=w_test)


print(f"  MAE : {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²  : {r2:.4f}\n")

Evaluación en train set:

  MAE : 0.0321
  RMSE: 0.0435
  R²  : 0.9559


Evaluación en test set:

  MAE : 0.0847
  RMSE: 0.1138
  R²  : 0.6658



### 3.1.2 Random forest tuneando hiperparámetros

In [12]:
# ========= Definimos grilla de hiperparámetros y entrenamos  ========= #

param_grid = {
    'n_estimators': [200,500,800],            # bajo, medio, alto
    'max_depth': [10,20,30],                  # poco profundo, medio, más profundo
    'min_samples_leaf': [1],                  # fijo (default)
    'min_samples_split': [4, 6],              # Permite divisiones del nodo cuando tienen 100/ 200 alumnos aprox
    'min_weight_fraction_leaf': [0.0002],     # dos colegios aprox (400.000*0.0001= 
    'max_features': [0.3, 0.5,'sqrt'],             # intermedio + regularización fuerte
    #'max_leaf_nodes': [500,400],             # Controlado on max_dept (son nodos terminales) 
    'max_samples': [0.6,0.7, 0.8],            # árboles con menos o más datos
    #'ccp_alpha': [0.0001, 0.001],            # poda suave
    'bootstrap': [True],                      # fijo
    'criterion': ['squared_error'],           # fijo
    'oob_score': [False]                      # fijo
}

model = RandomForestRegressor(random_state=1, n_jobs=1)

#Búsqueda por grid search de la mejor combinación de parámetros
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=1,
    n_jobs=-1
)

#Entrenamiento
grid_search.fit(X_train_prep, y_train['porcentaje_satisfactorio'], sample_weight=w_train)


Fitting 5 folds for each of 162 candidates, totalling 810 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=1, random_state=1),
             n_jobs=-1,
             param_grid={'bootstrap': [True], 'criterion': ['squared_error'],
                         'max_depth': [10, 20, 30],
                         'max_features': [0.3, 0.5, 'sqrt'],
                         'max_samples': [0.6, 0.7, 0.8],
                         'min_samples_leaf': [1], 'min_samples_split': [4, 6],
                         'min_weight_fraction_leaf': [0.0002],
                         'n_estimators': [200, 500, 800],
                         'oob_score': [False]},
             scoring='neg_mean_absolute_error', verbose=1)

In [13]:
# ========= Seteamos el modelo con los parámetros del mejor modelo obtenido ========= #
best_model = grid_search.best_estimator_

In [14]:
# ========= Predecimos en el conjunto de train y de test  ========= #
y_train_pred = best_model.predict(X_train_prep)
y_test_pred = best_model.predict(X_test_prep)

In [15]:
# ========= Evaluamos la predicción  ========= #
print("Evaluación en train set:\n")
mae = mean_absolute_error(y_train_prep, y_train_pred, sample_weight=w_train)
rmse = np.sqrt(mean_squared_error(y_train_prep, y_train_pred, sample_weight=w_train))
r2 = r2_score(y_train_prep, y_train_pred, sample_weight=w_train)

print(f"  MAE : {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²  : {r2:.4f}\n")

print("")

print("Evaluación en test set:\n")

mae = mean_absolute_error(y_test_prep, y_test_pred, sample_weight=w_test)
rmse = np.sqrt(mean_squared_error(y_test_prep, y_test_pred, sample_weight=w_test))
r2 = r2_score(y_test_prep, y_test_pred, sample_weight=w_test)


print(f"  MAE : {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²  : {r2:.4f}\n")

Evaluación en train set:

  MAE : 0.0407
  RMSE: 0.0581
  R²  : 0.9214


Evaluación en test set:

  MAE : 0.0841
  RMSE: 0.1128
  R²  : 0.6716



## 3.2 XGBoost con pesos (w=número alumnos que respondienron el examen)

### 3.2.1 XGBoost hiperparámetros por default

In [16]:
# ========= Iniciamos el modelo y entrenamos  ========= #
xgb_model = XGBRegressor(n_jobs=-1, random_state=1)
xgb_model.fit(X_train_prep, y_train_prep, sample_weight=w_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=-1, num_parallel_tree=None, ...)

In [17]:
# ========= Predecimos en el conjunto de train y de test  ========= #
y_train_pred = xgb_model.predict(X_train_prep)
y_test_pred = xgb_model.predict(X_test_prep)

In [18]:
# ========= Evaluamos la predicción  ========= #
print("Evaluación en train set:\n")
mae = mean_absolute_error(y_train_prep, y_train_pred, sample_weight=w_train)
rmse = np.sqrt(mean_squared_error(y_train_prep, y_train_pred, sample_weight=w_train))
r2 = r2_score(y_train_prep, y_train_pred, sample_weight=w_train)

print(f"  MAE : {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²  : {r2:.4f}\n")

print("")

print("Evaluación en test set:\n")

mae = mean_absolute_error(y_test_prep, y_test_pred, sample_weight=w_test)
rmse = np.sqrt(mean_squared_error(y_test_prep, y_test_pred, sample_weight=w_test))
r2 = r2_score(y_test_prep, y_test_pred, sample_weight=w_test)


print(f"  MAE : {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²  : {r2:.4f}\n")

Evaluación en train set:

  MAE : 0.0212
  RMSE: 0.0278
  R²  : 0.9819


Evaluación en test set:

  MAE : 0.0864
  RMSE: 0.1157
  R²  : 0.6550



### 3.2.2XGBoost tunenado hiperparámetros con pesos (número alumnos)

Dividimos la muestra en train y test. Luego, convertimos a objeto a las variables categóricas (4 en total, jurisdicción, clima escolar, region y sector) para luego hacer el one hot encoding.

In [19]:
# ========= Definimos grilla de hiperparámetros y entrenamos  ========= #
############ Esta es la opción 1. Usamos la 2, ver README
param_grid_xgb = {
    'n_estimators': [200, 500],
    'max_depth': [6, 10, 14],
    'learning_rate': [0.05],
    'min_child_weight': [200],
    'subsample': [0.6, 0.7],
    'colsample_bytree': [0.3, 0.5],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [1],
    'gamma': [0.05],
    'booster': ['gbtree'],
    'objective': ['reg:squarederror'],
}


########### Modelo base ###########
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    random_state=1,
    n_jobs=-1
)

########### Búsqueda con GridSearch ###########
xgb_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=0,   
    n_jobs=-1
)

# ========= Esto es para ver el progreso, se puede remover ========= #
@contextmanager
def tqdm_joblib(tqdm_object):
    """Muestra progreso de joblib en una barra tqdm."""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)
    old_cb = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_cb
        tqdm_object.close()


n_candidates = len(list(ParameterGrid(param_grid_xgb)))   
n_splits = xgb_search.cv if isinstance(xgb_search.cv, int) else xgb_search.cv.get_n_splits(
    X_train_prep, y_train['porcentaje_satisfactorio'])
total_fits = n_candidates * n_splits + (1 if getattr(xgb_search, "refit", True) else 0)


# ========= ENTRENAMIENTO (ÚNICO fit, ENVUELTO) ========= #
with tqdm_joblib(tqdm(total=total_fits, desc="GridSearchCV", unit="fit")):
    xgb_search.fit(X_train_prep, y_train['porcentaje_satisfactorio'], sample_weight=w_train)

GridSearchCV:   0%|          | 0/241 [00:00<?, ?fit/s]

In [20]:
# ========= Seteamos el modelo con los parámetros del mejor modelo obtenido y luego entrenamos  ========= #
best_xgb = xgb_search.best_estimator_

In [21]:
# ========= Imprimir parámetros  ========= #

best_xgb.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.3,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'feature_weights': None,
 'gamma': 0.05,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.05,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 6,
 'max_leaves': None,
 'min_child_weight': 200,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 200,
 'n_jobs': -1,
 'num_parallel_tree': None,
 'random_state': 1,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.7,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [22]:
# ========= Predecimos en el conjunto de train y de test  ========= #
y_train_pred = best_xgb.predict(X_train_prep)
y_test_pred = best_xgb.predict(X_test_prep)

In [23]:
# ========= Evaluamos la predicción  ========= #
print("Evaluación en train set:\n")
mae = mean_absolute_error(y_train_prep, y_train_pred, sample_weight=w_train)
rmse = np.sqrt(mean_squared_error(y_train_prep, y_train_pred, sample_weight=w_train))
r2 = r2_score(y_train_prep, y_train_pred, sample_weight=w_train)

print(f"  MAE : {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²  : {r2:.4f}\n")

print("")

print("Evaluación en test set:\n")

mae = mean_absolute_error(y_test_prep, y_test_pred, sample_weight=w_test)
rmse = np.sqrt(mean_squared_error(y_test_prep, y_test_pred, sample_weight=w_test))
r2 = r2_score(y_test_prep, y_test_pred, sample_weight=w_test)


print(f"  MAE : {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²  : {r2:.4f}\n")

Evaluación en train set:

  MAE : 0.0519
  RMSE: 0.0698
  R²  : 0.8865


Evaluación en test set:

  MAE : 0.0793
  RMSE: 0.1077
  R²  : 0.7009



In [24]:
#### Esta es op 2 y la que finalmente usamos
# ========= Definimos grilla de hiperparámetros y entrenamos  ========= #
param_grid_xgb = {
    'n_estimators': [200, 500],
    'max_depth': [6, 10, 20],
    'learning_rate': [0.01,0.05],
    'min_child_weight': [150,300],
    'subsample': [0.6, 0.7],
    'colsample_bytree': [0.3, 0.5],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [1],
    'gamma': [0.005],
    'booster': ['gbtree'],
    'objective': ['reg:squarederror'],
}
########### Modelo base ###########
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    random_state=1,
    n_jobs=-1
)

########### Búsqueda con GridSearch ###########
xgb_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=0,   # silenciado para no ensuciar la barra
    n_jobs=-1
)

# ========= BLOQUE DE PROGRESO (antes del fit) ========= #
@contextmanager
def tqdm_joblib(tqdm_object):
    """Muestra progreso de joblib en una barra tqdm."""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)
    old_cb = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_cb
        tqdm_object.close()


n_candidates = len(list(ParameterGrid(param_grid_xgb)))   
n_splits = xgb_search.cv if isinstance(xgb_search.cv, int) else xgb_search.cv.get_n_splits(
    X_train_prep, y_train['porcentaje_satisfactorio'])
total_fits = n_candidates * n_splits + (1 if getattr(xgb_search, "refit", True) else 0)


# ========= ENTRENAMIENTO (ÚNICO fit, ENVUELTO) ========= #
with tqdm_joblib(tqdm(total=total_fits, desc="GridSearchCV", unit="fit")):
    xgb_search.fit(X_train_prep, y_train['porcentaje_satisfactorio'], sample_weight=w_train)

GridSearchCV:   0%|          | 0/961 [00:00<?, ?fit/s]

In [25]:
# ========= Seteamos el modelo con los parámetros del mejor modelo obtenido y luego entrenamos  ========= #
best_xgb = xgb_search.best_estimator_

In [26]:
# ========= Predecimos en el conjunto de train y de test  ========= #
y_train_pred = best_xgb.predict(X_train_prep)
y_test_pred = best_xgb.predict(X_test_prep)

In [27]:
# ========= Evaluamos la predicción  ========= #
print("Evaluación en train set:\n")
mae = mean_absolute_error(y_train_prep, y_train_pred, sample_weight=w_train)
rmse = np.sqrt(mean_squared_error(y_train_prep, y_train_pred, sample_weight=w_train))
r2 = r2_score(y_train_prep, y_train_pred, sample_weight=w_train)

print(f"  MAE : {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²  : {r2:.4f}\n")

print("")

print("Evaluación en test set:\n")

mae = mean_absolute_error(y_test_prep, y_test_pred, sample_weight=w_test)
rmse = np.sqrt(mean_squared_error(y_test_prep, y_test_pred, sample_weight=w_test))
r2 = r2_score(y_test_prep, y_test_pred, sample_weight=w_test)


print(f"  MAE : {mae:.5f}")
print(f"  RMSE: {rmse:.5f}")
print(f"  R²  : {r2:.5f}\n")

Evaluación en train set:

  MAE : 0.0529
  RMSE: 0.0719
  R²  : 0.8794


Evaluación en test set:

  MAE : 0.08007
  RMSE: 0.10786
  R²  : 0.69993



### 3.2.3 SHAP values

**Paso a paso**


**Paso 1**

data=X_train_prep. SHAP lo usa para dos cosas:
* Calcular el valor base (E[f(X)]) como el promedio de las predicciones del modelo sobre estos datos
* Saber cómo se distribuyen las variables al calcular qué pasa cuando se “ocultan” variables:
    
    Supongamos que tenés una observación así: NSE = alto, infraestructura = media, clima_escolar = bueno, Y querés ocultar clima_escolar. ¿Qué hace SHAP? Mantiene NSE e infraestructura como están. En lugar de usar el valor real de clima_escolar, lo reemplaza por "lo que sabe del clima escolar cuando no conocemos esta observación específica".

* model_output='raw'. Especifica que el modelo devuelve directamente el valor de predicción. Esto es el default para regresión, así que es correcto.

feature_perturbation='tree_path_dependent'. Le dice a SHAP que use la lógica interna de los árboles para calcular los valores cuando se “ocultan” variables. Esto es más eficiente y exacto para modelos de árbol. Evita la aleatoriedad de sampling.



In [28]:
# ========= Shap values ========= #
explainer = shap.TreeExplainer(model=best_xgb, data=X_train_prep, model_output='raw', feature_perturbation='interventional')

In [29]:
shap_values_train_xgb = explainer.shap_values(X_train_prep)

 99%|===================| 9029/9085 [02:09<00:00]        

In [30]:
shap_values_test_xgb = explainer.shap_values(X_test_prep)

100%|===================| 2265/2272 [00:33<00:00]        

In [31]:
shap_df_train_xgb = pd.DataFrame(shap_values_train_xgb, index=X_train_prep.index, columns=X_train_prep.columns)
shap_df_test_xgb = pd.DataFrame(shap_values_test_xgb, index=X_test_prep.index, columns=X_test_prep.columns)

In [32]:
print("expected(test background):", explainer.expected_value)
print("mean(y_pred_test):", y_train_pred.mean())
print("mean SHAP sum on test (should be ~0):", shap_values_train_xgb.sum(axis=1).mean())

expected(test background): 0.4580926612953473
mean(y_pred_test): 0.46310946
mean SHAP sum on test (should be ~0): 0.005016720318804907


In [33]:
# ========= Agregamos predicciones y valores reales a los shaps ========= #

##Aseguramos alineación de  y_pred
#Train
y_train_pred_s = pd.Series(y_train_pred, index=X_train_prep.index, name="prediccion")
shap_df_train_xgb = shap_df_train_xgb.join(y_train_pred_s)
shap_df_train_xgb = shap_df_train_xgb.join(y_train["porcentaje_satisfactorio"])

#Test
y_test_pred_s = pd.Series(y_test_pred, index=X_test_prep.index, name="prediccion")
shap_df_test_xgb = shap_df_test_xgb.join(y_test_pred_s)
shap_df_test_xgb = shap_df_test_xgb.join(y_test["porcentaje_satisfactorio"])


In [34]:
# ========= Agregamos info adicional de los colegios ========= #
shap_df_train_xgb = shap_df_train_xgb.join(info_extra, how='left')
shap_df_test_xgb = shap_df_test_xgb.join(info_extra, how='left')


shap_df_train_xgb = shap_df_train_xgb.reset_index()
shap_df_test_xgb = shap_df_test_xgb.reset_index()


In [35]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

# Armar path con fecha
filename = f"../outputs/shap_df_xgb_{today}_old_column_order_new_datacleaning_script.xlsx"

with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
    shap_df_test_xgb.to_excel(writer, sheet_name='SHAP_Test', index=False)
    shap_df_train_xgb.to_excel(writer, sheet_name='SHAP_Train', index=False)


In [36]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

# Armar path con fecha
filename = f"../outputs/shap_df_xgb_{today}.xlsx"

with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
    shap_df_test_xgb.to_excel(writer, sheet_name='SHAP_Test', index=False)
    shap_df_train_xgb.to_excel(writer, sheet_name='SHAP_Train', index=False)